# Starting the Environment
1. Open an Anaconda Prompt terminal,
   - On windows, Search `anaconda prompt`, click to open
2. Change directory (`cd`) into class directory on your computer,
   - `cd C:\JW\Clarkson\AE470`
3. Change directory into the repository on your computer,
   - `cd AE470_Sp25`
4. Fetch the latest class repository from GitHub,
   - **This will overwrite any changes you have made to files in your local repository directory, `AE470_Sp2025`.**
   - **Be sure to rename any files where you make changes that you want to keep.**
   - `git fetch origin`
5. Reset your local branch repository to match the remote branch,
   - `git reset --hard origin/main`
6. Activate the virtual python environment,
   - `conda activate ae470sp25`
7. Start a Jupyter notebook session in a browser window.  Type the following into an Anaconda Prompt window,
   - `jupyter notebook`
8. Using the Jupyter browser, open this notebook: `03_ae470_constants_of_motion.ipynb`.


In [ ]:
# Configure Jupyter so figures appear in the notebook
# %matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [ ]:
# import modsim and install if necessary
try:
    from modsim import *
except ImportError:
    print("Download modsim from the course repository at https://github.com/jeffwalton/AE470_Sp25")

# Constants of the Motion

## Angular Momentum

We define *orbital angular momentum* (or *specific angular momentum* - the angular momentum per unit mass) as 
\begin{equation}
\vec{\mathbf{h}} \triangleq \vec{\mathbf{r}} \times \vec{\mathbf{v}} 
\end{equation}

### By Numerical Simulation

Looking at the simulation from the previous section, we will calculate the angular momentum vector and its magnitude.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Re-run the simulation in this notebook.

In [ ]:
r0 = Vector( 8000, 0, 6000 ) # km
v0 = Vector( 0, 7, 0 ) # km/s

init = State(x=r0.x, y=r0.y, z=r0.z, vx=v0.x, vy=v0.y, vz=v0.z)

system = System(init=init,
                mu=3.986000e+5, # km^3/s^2
                t_end=4 * 60 * 60, # s
               )

show(system)

In [ ]:
def eom_twobodies(t, state, system):
    x, y, z, vx, vy, vz = state  # unpack the state vector
    mu = system.mu
    
    R = Vector(x, y, z)
    r = vector_mag(R)
    V = Vector(vx, vy, vz)
    # print(f"time:{t}, R:[{R.x}, {R.y}, {R.z}]")

    R_dot = V
    R_dotdot = -mu / r**3 * R

    return R_dot.x, R_dot.y, R_dot.z, R_dotdot.x, R_dotdot.y, R_dotdot.z


In [ ]:
results, details = run_solve_ivp(system, eom_twobodies, first_step=10.0, rtol=1e-5)

details.message

In [ ]:
results

Define a python function to compute the magnitude of angular momentum, $ |\vec{\mathbf{h}}| $.

In [ ]:
def ang_momentum(r_x, r_y, r_z, v_x, v_y, v_z):
    R = Vector(r_x, r_y, r_z)
    V = Vector(v_x, v_y, v_z)
    H = vector_cross(R,V)
    return vector_mag(H)

We can add a column to the results DataFrame for `h_mag`. To do this we need to apply the the `ang_momentum()` function to each row of `results`. This is done through the [pandas.DataFrame.apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method. Take a look at the next cell. In python, `lambda` is a special in-line function.  You can learn more about lambda functions in the [Real Python](https://realpython.com/) article, [How to Use Python Lambda Functions](https://realpython.com/python-lambda/). In our case, `lambda` allows us to access individual columns of `results` for a specific row. The parameter `axis=1` applies the function to each row. The resulting value is stored in `results` in a column named **h_mag**.

In [ ]:
 results['h_mag'] = results.apply(lambda row: ang_momentum(row.x, row.y, row.z, row.vx, row.vy, row.vz), axis=1)

In [ ]:
results

Let's make a plot of $ |\vec{\mathbf{h}}| $ for the whole 4 hours.

In [ ]:
results.h_mag.plot()
decorate(xlabel='Time (s)', ylabel='h_mag')

In [ ]:
print(f"The minimum and maximum values of h_mag are: min: {results.h_mag.min()}, max: {results.h_mag.max()}")

In [ ]:
results.h_mag.std()

The magnitude of $ \vec{\mathbf{h}} $ is constant to the precision of the numerical integration.

### Analytically

Take the inertial time-derivative of $ \vec{\mathbf{h}} = \vec{\mathbf{r}} \times \vec{\mathbf{v}}$,
\begin{align}
\dot{\vec{\mathbf{h}}} &= \dot{\vec{\mathbf{r}}} \times \vec{\mathbf{v}} + \vec{\mathbf{r}} \times \dot{\vec{\mathbf{v}}} \\
&= \vec{\mathbf{v}} \times \vec{\mathbf{v}} + \vec{\mathbf{r}} \times \ddot{\vec{\mathbf{r}}} \\
&= \vec{\mathbf{0}} + \vec{\mathbf{r}} \times \ddot{\vec{\mathbf{r}}} \\
\end{align}

Substitute the equation of motion for $ \ddot{\vec{\mathbf{r}}} $

\begin{align}
\dot{\vec{\mathbf{h}}} &= \vec{\mathbf{r}} \times \bigg(-\frac{\mu}{|\vec{\mathbf{r}}|^3}\vec{\mathbf{r}} \bigg)\\
&= -\frac{\mu}{|\vec{\mathbf{r}}|^3}(\vec{\mathbf{r}} \times \vec{\mathbf{r}} )\\
&= \vec{\mathbf{0}} \\
\end{align}

Therefore, the orbital angular momentum is constant; it is conserved.  

Because $ \vec{\mathbf{h}} \triangleq \vec{\mathbf{r}} \times \vec{\mathbf{v}} $ and the definition of the vector cross product, $ \vec{\mathbf{r}} $ and $ \vec{\mathbf{v}} $ are always perpendicular to $ \vec{\mathbf{h}} $.  Since $ \vec{\mathbf{h}} $ is constant, $ \vec{\mathbf{r}} $ and $ \vec{\mathbf{v}} $ remain in the same plane.  This plane is called the *orbital plane*.  **Two-body orbital motion is planar.**

### Exercises
 1. What are the units of $ \vec{\mathbf{h}} $?  What does this say about the orbit?

## Orbital Energy

The *orbital energy* is defined as 
\begin{equation}
\mathcal{E} \triangleq \frac{v^2}{2} - \frac{\mu}{r} = \frac{\vec{\mathbf{v}} \cdot \vec{\mathbf{v}}}{2} - \frac{\mu}{r} 
\end{equation}
where $ v = |\vec{\mathbf{v}}| $ is the orbital speed and $ r = |\vec{\mathbf{r}}| $ is the distance of $m_2$ from $m_1$.  The term $\frac{v^2}{2}$ is the *kinetic energy per unit mass*. $-\frac{\mu}{r}$ is the *two-body gravitational potential energy per unit mass*.


### By Numerical Simulation

Looking at the simulation from the previous section, calculate the orbital energy.


In [ ]:
def orbital_energy(r_x, r_y, r_z, v_x, v_y, v_z, system):
    r = vector_mag(Vector(r_x, r_y, r_z))
    v = vector_mag(Vector(v_x, v_y, v_z))
    mu = system.mu
    return (v**2)/2 - mu/r

In [ ]:
 results['E'] = results.apply(lambda row: orbital_energy(row.x, row.y, row.z, row.vx, row.vy, row.vz, system), axis=1)

In [ ]:
results

Make of plot of $ \mathcal{E}$.

In [ ]:
results.E.plot()
decorate(xlabel='Time (s)', ylabel='$\mathcal{E}$')

In [ ]:
print(f"The minimum and maximum values of E are: min: {results.E.min()}, max: {results.E.max()}")


In [ ]:
results.E.std()

The orbital energy, $\mathcal{E}$, is constant to the precision of the numerical integration.

### Analytically

Compute time-derivative of 
\begin{equation}
\mathcal{E} \triangleq \frac{v^2}{2} - \frac{\mu}{r} = \frac{\vec{\mathbf{v}} \cdot \vec{\mathbf{v}}}{2} - \frac{\mu}{r} 
\end{equation}

\begin{align}
\dot{\mathcal{E}} &= \vec{\mathbf{v}} \cdot \dot{\vec{\mathbf{v}}}   + \frac{\dot{r}\mu}{r^2}  \\
\end{align}

From the dot product, $ r^2 = \vec{\mathbf{r}} \cdot \vec{\mathbf{r}} $

Taking the derivative, $ 2r\dot{r} = \vec{\mathbf{r}} \cdot \dot{\vec{\mathbf{r}}} + \dot{\vec{\mathbf{r}}} \cdot \vec{\mathbf{r}} = 2(\vec{\mathbf{r}} \cdot \dot{\vec{\mathbf{r}}}) \implies r\dot{r} = \vec{\mathbf{r}} \cdot \dot{\vec{\mathbf{r}}} \implies \dot{r} = \frac{\vec{\mathbf{r}} \cdot \vec{\mathbf{v}}}{r} $ 

Substituting $\dot{r}$ into $ \dot{\mathcal{E}} $ above.

\begin{align}
\dot{\mathcal{E}} &= \vec{\mathbf{v}} \cdot \dot{\vec{\mathbf{v}}}   + \frac{\mu}{r^3}  \vec{\mathbf{r}} \cdot \vec{\mathbf{v}}  \\
\end{align}

and substituting the two-body equation of motion for $ \dot{\vec{\mathbf{v}}} ( = \ddot{\vec{\mathbf{r}}} ) $

\begin{align}
\dot{\mathcal{E}} &= \vec{\mathbf{v}} \cdot ( - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}} ) + \frac{\mu}{r^3}  \vec{\mathbf{r}} \cdot \vec{\mathbf{v}}  \\
\dot{\mathcal{E}} &= 0
\end{align}


**For the two-body problem, orbital energy is constant.**

## The Eccentricity Vector

This constant of motion is not initially obvious.

Starting from the two-body equation of motion, $ \ddot{\vec{\mathbf{r}}} = - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}} $, take the cross product of both sides with the angular momentum $ \vec{\mathbf{h}} $.

\begin{equation} 
\ddot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} = - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}} \times \vec{\mathbf{h}}
\end{equation}

Since $ \frac{d }{d t}(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}}) = \ddot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} + \dot{\vec{\mathbf{r}}} \times \dot{\vec{\mathbf{h}}} $, the left-hand side can be written as

\begin{equation}
\ddot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} = \frac{d }{d t}(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}}) - \dot{\vec{\mathbf{r}}} \times \dot{\vec{\mathbf{h}}}
\end{equation}

We know that the angular momentum is constant ($ \dot{\vec{\mathbf{h}}} = \vec{\mathbf{0}} $), so

\begin{equation}
\ddot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} = \frac{d }{d t}(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}})
\end{equation}

The right-hand side can be reduced with the definition of $ \vec{\mathbf{h}} = \vec{\mathbf{r}} \times \vec{\mathbf{v}} $, then apply the bac-cab rule, $ \vec{a} \times (\vec{b} \times \vec{c}) = \vec{b}(\vec{a} \cdot \vec{c}) - \vec{c} (\vec{a} \cdot \vec{b}) $. 
\begin{align}
\frac{1}{r^3} \vec{\mathbf{r}} \times \vec{\mathbf{h}} &= \frac{1}{r^3} ( \vec{\mathbf{r}} \times (\vec{\mathbf{r}} \times \vec{\mathbf{v}}) ) \\
 &= \frac{1}{r^3} \bigg( \vec{\mathbf{r}} (\vec{\mathbf{r}} \cdot \vec{\mathbf{v}}) - \vec{\mathbf{v}} (\vec{\mathbf{r}} \cdot \vec{\mathbf{r}}) \bigg) \\
 &= \frac{1}{r^3} \bigg( \vec{\mathbf{r}} (r \dot{r}) - \vec{\mathbf{v}} r^2 \bigg) \\
 &= \frac{\vec{\mathbf{r}} \dot{r} - \vec{\mathbf{v}} r}{r^2} \\
\end{align}

Consider now the time derivative of the unit vector $ \hat{\mathbf{r}} = \frac{\vec{\mathbf{r}}}{r} $,
\begin{equation}
\frac{d}{d t}\bigg(\frac{\vec{\mathbf{r}}}{r} \bigg) =  \frac{d}{d t}\bigg( \vec{\mathbf{r}} \frac{1}{r} \bigg) = \dot{\vec{\mathbf{r}}} \frac{1}{r} - \frac{\dot{r}}{r^2} \vec{\mathbf{r}}  = \frac{r \dot{\vec{\mathbf{r}}} - \vec{\mathbf{r}} \dot{r} }{r^2} = - \frac{\vec{\mathbf{r}} \dot{r} - r \dot{\vec{\mathbf{r}}} }{r^2} = - \frac{\vec{\mathbf{r}} \dot{r} - r \vec{\mathbf{v}} }{r^2}
\end{equation}

Therefore, 
\begin{align}
\frac{1}{r^3} \vec{\mathbf{r}} \times \vec{\mathbf{h}} &= - \frac{d}{d t}\bigg(\frac{\vec{\mathbf{r}}}{r} \bigg)
\end{align}


Substituting back into 
\begin{equation} 
\ddot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} = - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}} \times \vec{\mathbf{h}}
\end{equation}

we get
\begin{equation} 
\frac{d }{d t}(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}}) = \frac{d}{d t}\bigg(\mu \frac{\vec{\mathbf{r}}}{r} \bigg)
\end{equation}

or
\begin{equation} 
\frac{d }{d t}\bigg(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} - \mu \frac{\vec{\mathbf{r}}}{r} \bigg) = \vec{\mathbf{0}}
\end{equation}

This vector is constant
\begin{equation} 
\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} - \mu \frac{\vec{\mathbf{r}}}{r} = \vec{\mathbf{C}}
\end{equation} 

$ \vec{\mathbf{C}} $ is called the Laplace vector named after [Pierre-Simon Laplace (1749-1827)](https://en.wikipedia.org/wiki/Pierre-Simon_Laplace).

Taking the dot product of both sides with $ \vec{\mathbf{h}} $ yields

\begin{equation} 
(\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}}) \cdot \vec{\mathbf{h}} - \mu \frac{ \vec{\mathbf{r}} \cdot \vec{\mathbf{h}} }{r} = \vec{\mathbf{C}} \cdot \vec{\mathbf{h}}
\end{equation} 

Note that $ \dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} $ is perpendicular to both $ \dot{\vec{\mathbf{r}}} $ and $ \vec{\mathbf{h}} $, so $ (\dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}}) \cdot \vec{\mathbf{h}} = 0 $.  Similarlly, because $ \vec{\mathbf{h}} = \vec{\mathbf{r}} \times \dot{\vec{\mathbf{r}}} $ is perpendicular to both $ \vec{\mathbf{r}} $ and $ \dot{\vec{\mathbf{r}}} $, it is evident that $ \vec{\mathbf{r}} \cdot \vec{\mathbf{h}} = 0 $.  Therefore, $ \vec{\mathbf{C}} \cdot \vec{\mathbf{h}} = 0 $; $ \vec{\mathbf{C}} $ is perpendicular to $ \vec{\mathbf{h}} $ and lies in the orbital plane.

Rearranging the Laplace vector equation and defining $ \vec{\mathbf{e}} = \vec{\mathbf{C}} / \mu$,
\begin{equation} 
\vec{\mathbf{e}} \triangleq \frac{ \dot{\vec{\mathbf{r}}} \times \vec{\mathbf{h}} }{\mu} - \frac{ \vec{\mathbf{r}}}{r}
\end{equation} 

$ \vec{\mathbf{e}} $ is called the *eccentricity vector*.  It is dimensionless and is fixed in the orbital plane.

### By Numerical Simulation

Looking at the simulation from the previous section, calculate the magnitude of $ \vec{\mathbf{e}} $.


In [ ]:
def eccentricity_vector_mag(r_x, r_y, r_z, v_x, v_y, v_z, system):
    R = Vector(r_x, r_y, r_z)
    r = vector_mag(R)
    V = Vector(v_x, v_y, v_z)
    H = vector_cross(R,V)
    mu = system.mu
    e_vec = (1/mu)*vector_cross(V,H) - (1/r)*R
    return vector_mag(e_vec)    

In [ ]:
results['ecc'] = results.apply(lambda row: eccentricity_vector_mag(row.x, row.y, row.z, row.vx, row.vy, row.vz, system), axis=1)

In [ ]:
results

In [ ]:
results.ecc.plot()
decorate(xlabel='Time (s)', ylabel='ecc')

In [ ]:
print(f"The minimum and maximum values of e are: min: {results.ecc.min()}, max: {results.ecc.max()}")

e is constant to three decimal places.

## References

    R.R. Bate, D.D. Mueller, J.E. White, W.W. Saylor; Fundamentals of Astrodynamics, Dover, 2nd Ed, 2020.  

    H.D. Curtis; Orbital Mechanics for Engineering Students; Butterworth-Heinemann, 4th Ed, 2021.  

    A.H.J. de Ruiter, C.J. Damaren, J.R. Forbes; Spacecraft Dynamics and Control: An Introduction; Wiley, 2013.

